**Feedforward Neural Network (FNN)**

Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gc
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import os
import pandas as pd
import random

Load csv into df

In [2]:
# --- Ensure consistent working directory for data loading ---
# This block dynamically sets the current working directory to the Git repository root.
# This makes data paths reliable for all collaborators, regardless of where they open the notebook.

current_dir = os.getcwd()
repo_root = current_dir
while not os.path.exists(os.path.join(repo_root, '.git')):
    # Move up one directory
    parent_dir = os.path.dirname(repo_root)
    if parent_dir == repo_root: # Reached filesystem root, .git not found
        raise FileNotFoundError(
            "Could not find the .git directory. "
            "Please ensure you are running this code from within a Git repository."
        )
    repo_root = parent_dir

# Change the current working directory if it's not already the repo root
if os.getcwd() != repo_root:
    os.chdir(repo_root)
    print(f"Working directory set to: {os.getcwd()}") # Informative print for users


# --- Data Loading ---
# Path to the data file, relative to the repository root.
data_file_name = 'Customer_Purchasing_Behaviors.csv'
data_file_path = os.path.join('src', 'data', data_file_name)

try:
    df = pd.read_csv(data_file_path)
    print(f"Successfully loaded '{data_file_name}' into the DataFrame named df.")
    #print(df.head())
except FileNotFoundError:
    print(f"Error: The file '{data_file_name}' was not found at '{data_file_path}'.")
    print("Please ensure it exists in the 'src/data/' folder relative to the repository root.")
except Exception as e:
    print(f"An error occurred during data loading: {e}")

Working directory set to: c:\Users\The Winner\DSI\customer_purchasing_behaviour
Successfully loaded 'Customer_Purchasing_Behaviors.csv' into the DataFrame named df.


Prepare data: scale numerical and encode categorical

In [3]:
# Preprocessing: Prepare Inputs

# Extract relevant columns
numerical_cols = ['age', 'annual_income', 'loyalty_score', 'purchase_frequency']
categorical_col = 'region'
target_col = 'purchase_amount'

# Scale numerical features
scaler = StandardScaler()
X_num = scaler.fit_transform(df[numerical_cols])

# Encode categorical feature
encoder = OneHotEncoder(sparse_output=False)
X_cat = encoder.fit_transform(df[[categorical_col]])

# Create DataFrames with distinct column names
num_df = pd.DataFrame(X_num, columns=numerical_cols)
cat_columns = encoder.get_feature_names_out([categorical_col])
cat_df = pd.DataFrame(X_cat, columns=cat_columns)

# Combine safely
combined_df = pd.concat([num_df, cat_df], axis=1)

# Convert to torch tensor
X_combined = torch.tensor(combined_df.values, dtype=torch.float32)

# Target variable
y = torch.tensor(df[target_col].values, dtype=torch.float32).view(-1, 1)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

In [ ]:
# Make it reproducible

SEED = 42
random.seed(SEED)                 # Python random
np.random.seed(SEED)              # NumPy
torch.manual_seed(SEED)           # CPU
torch.cuda.manual_seed(SEED)      # GPU
torch.cuda.manual_seed_all(SEED)  # Multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define the Neural Network Model

class FeedforwardNN(nn.Module):
    def __init__(self, input_size):
        super(FeedforwardNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.layers(x)

model = FeedforwardNN(input_size=X_train.shape[1])
def init_weights(m):
    '''Make the weight initialization reproducible'''
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)  # still uses seed
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0.01)
model.apply(init_weights)      

# Make GPU work
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move data to device
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
# Move model to device
model = model.to(device)

# Training with MSE & Logging RMSE, MAE 

# Define RMSE metric (no need to backprop through it)
def compute_rmse(predictions, targets):
    return torch.sqrt(F.mse_loss(predictions, targets))
def compute_mae(predictions, targets):
    return torch.mean(torch.abs(predictions - targets))

# Train the Model

criterion = nn.MSELoss()            # Mean Squared Error (MSE) loss
optimizer = optim.Adam(model.parameters(), lr=0.005)

for epoch in range(10001):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)       # MSE for optimization
    loss.backward()
    optimizer.step()
    # Log RMSE & MAE every N epoch
    if epoch % 1000 == 0:
        rmse = compute_rmse(output, y_train)
        mae = compute_mae(output, y_train)
        print(f"Epoch {epoch}: Losses: MSE = {loss.item():.4f}, RMSE = {rmse.item():.4f}, MAE = {mae.item():.4f}")

# Evaluate the Model

model.eval()
with torch.no_grad():
    predictions = model(X_test)
    test_loss = criterion(predictions, y_test)
    test_rmse = compute_rmse(predictions, y_test)
    test_mae = compute_mae(predictions, y_test)
    print(f"Test:        Losses: MSE = {test_loss.item():.4f}, RMSE = {test_rmse.item():.4f}, MAE = {test_mae.item():.4f}")

# Clean memory in case we want to run this cell again without running the whole notebook
# remove references to GPU objects 
del model
# Invoke garbage collector
gc.collect()
# Clear GPU cache
torch.cuda.empty_cache()

Epoch 0: Losses: MSE = 202057.7500, RMSE = 449.5083, MAE = 428.2060
Epoch 1000: Losses: MSE = 99.3161, RMSE = 9.9657, MAE = 7.2834
Epoch 2000: Losses: MSE = 92.3433, RMSE = 9.6095, MAE = 7.0018
Epoch 3000: Losses: MSE = 91.5156, RMSE = 9.5664, MAE = 6.9330
Epoch 4000: Losses: MSE = 91.4174, RMSE = 9.5612, MAE = 6.9101
Epoch 5000: Losses: MSE = 91.4091, RMSE = 9.5608, MAE = 6.9026
Epoch 6000: Losses: MSE = 91.4084, RMSE = 9.5608, MAE = 6.9005
Epoch 7000: Losses: MSE = 91.4087, RMSE = 9.5608, MAE = 6.9002
Epoch 8000: Losses: MSE = 91.4993, RMSE = 9.5655, MAE = 6.8886
Epoch 9000: Losses: MSE = 91.4085, RMSE = 9.5608, MAE = 6.9002
Epoch 10000: Losses: MSE = 91.4086, RMSE = 9.5608, MAE = 6.9001
Test:         Losses: MSE = 114.2079, RMSE = 10.6868, MAE = 7.4063
